<a href="https://colab.research.google.com/github/stoicelephant/DIDA340/blob/main/Copy_of_Final_Project_Dida_340.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %env KAGGLE_USERNAME=jamesgiuf
# %env KAGGLE_KEY=c886c47a2c9510261eb39b5a38d569b9

# !kaggle datasets download -d mouadjaouhari/bitcoin-hourly-ohclv-dataset

# !unzip /content/bitcoin-hourly-ohclv-dataset.zip


In [1]:
import pandas as pd
import datetime as dt
bitcoin = pd.read_csv('https://raw.githubusercontent.com/stoicelephant/DIDA340/refs/heads/main/bitcoin_csv_simple')

In [ ]:
# bitcoin.columns

Index(['UNIX_TIMESTAMP', 'DATETIME', 'OPEN', 'HIGH', 'CLOSE', 'LOW', 'VOLUME'], dtype='object')

In [2]:
bitcoin['dt'] = pd.to_datetime(bitcoin['DATETIME'])

In [ ]:
# bitcoin[['DATETIME','OPEN','HIGH','CLOSE','LOW']].head(1)

,DATETIME,OPEN,HIGH,CLOSE,LOW
0,2014-12-15 06:00:00.000,349.78,350.07,349.54,348.81


In [ ]:
# bitcoin.DATETIME[1][0:7]

'2014-12'

In [ ]:
# b1512 = bitcoin.loc[bitcoin.DATETIME.str.startswith('2015-12')]

In [ ]:
# b1512.reset_index(drop=True)
# b1512

,UNIX_TIMESTAMP,DATETIME,OPEN,HIGH,CLOSE,LOW,VOLUME,dt
8418,1448928000,2015-12-01 00:00:00.000,378.03,379.28,378.57,376.87,791941.04,2015-12-01 00:00:00
8419,1448931600,2015-12-01 01:00:00.000,378.57,378.72,376.29,376.20,705943.21,2015-12-01 01:00:00
8420,1448935200,2015-12-01 02:00:00.000,376.29,378.40,377.00,375.46,749345.02,2015-12-01 02:00:00
8421,1448938800,2015-12-01 03:00:00.000,377.00,377.87,377.42,375.75,940507.61,2015-12-01 03:00:00
8422,1448942400,2015-12-01 04:00:00.000,377.42,378.30,377.72,376.72,708502.27,2015-12-01 04:00:00
...,...,...,...,...,...,...,...,...
9157,1451588400,2015-12-31 19:00:00.000,430.77,431.14,430.58,429.99,440238.44,2015-12-31 19:00:00
9158,1451592000,2015-12-31 20:00:00.000,430.58,431.51,430.99,429.83,962480.14,2015-12-31 20:00:00
9159,1451595600,2015-12-31 21:00:00.000,430.99,433.54,432.57,430.95,879519.94,2015-12-31 21:00:00
9160,1451599200,2015-12-31 22:00:00.000,432.57,432.99,430.98,430.33,788517.51,2015-12-31 22:00:00


In [3]:
!pip install plotly

In [ ]:
# import plotly.graph_objects as go


# fig = go.Figure(data=[go.Candlestick(x=b1512['DATETIME'],
#                 open=b1512['OPEN'],
#                 high=b1512['HIGH'],
#                 low=b1512['LOW'],
#                 close=b1512['CLOSE'])])

In [6]:
maxt= bitcoin.UNIX_TIMESTAMP.max()
mint= bitcoin.UNIX_TIMESTAMP.min()
week_secs = 60*60*24*7

range_dict = {}
count = 0

while mint <= maxt:
  inputs = [mint, mint+week_secs]
  mint = mint+week_secs
  range_dict[count] = inputs
  count +=1

Now i need to split the df into subframes

In [7]:
df_dict = {}
count = 0

for week in range_dict:
  count +=1
  current_range = range_dict[week]
  lhs = current_range[0]
  rhs = current_range[1]
  new_df =  bitcoin.loc[(bitcoin.UNIX_TIMESTAMP <= rhs) & (bitcoin.UNIX_TIMESTAMP > lhs)]
  df_dict[week] = new_df


In [8]:
num_weeks = len(range_dict) #565

In [14]:
!pip install --upgrade kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.7/373.7 kB 12.4 MB/s eta 0:00:00


In [16]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 11.1 MB/s eta 0:00:0000:0100:01
  Using cached contourpy-1.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (355 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 9.0 MB/s eta 0:00:00:00:0100:01
  Using cached kiwisolver-1.4.9-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (1.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 10.9 MB/s eta 0:00:00


In [ ]:
import plotly.graph_objects as go
from PIL import Image
import matplotlib.pyplot as plt
import plotly.express as px

num_weeks = len(range_dict)

for i in range(num_weeks):
  current_df = df_dict[i]
  fig = go.Figure(data=[go.Candlestick(x=current_df['DATETIME'],
                                       open=current_df['OPEN'],
                                       high=current_df['HIGH'],
                                       low=current_df['LOW'],
                                       close=current_df['CLOSE'])])

  fig.update_yaxes(dtick=20, showgrid=True, gridcolor="lightgray")
  fig.write_image(f"fig_{i}.png")


Still trying to figure out a way on how to standardize thse images